In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

from histopatseg.visualization.visualization import plot_embeddings
from histopatseg.evaluation.utils import aggregate_tile_embeddings, custom_balanced_group_kfold
from histopatseg.evaluation.prototype_classifier import PrototypeClassifier

In [ ]:
project_dir = Path(".").resolve().parent
print(f"Project Directory: {project_dir}")

In [ ]:
embedding_file = project_dir / "data/processed/embeddings/lunghist700_20x_UNI2_embeddings.npz"
metadata  = pd.read_csv(project_dir / "/home/valentin/workspaces/histopatseg/data/processed/LungHist700_tiled/LungHist700_20x/metadata.csv").set_index("tile_id")
metadata.head()

In [ ]:
# Load the embeddings
data = np.load(embedding_file)
embeddings = data["embeddings"]
tile_ids = data["tile_ids"]
embedding_dim = data["embedding_dim"]

# Check if all embedding tile_ids are in the metadata index
missing_ids = [id for id in tile_ids if id not in metadata.index]
if missing_ids:
    print(f"Warning: {len(missing_ids)} tile_ids from embeddings are not in metadata")
    print(f"First few missing IDs: {missing_ids[:5]}")

metadata = metadata.reindex(tile_ids)
metadata["subclass"] = metadata.apply(
    lambda row: row["superclass"]
    if pd.isna(row["subclass"]) and row["superclass"] == "nor"
    else row["subclass"],
    axis=1,
)
 

# Print basic information
print(f"Loaded {len(embeddings)} embeddings with dimensionality {embeddings.shape[1]}")
print(f"Embedding dimension from model: {embedding_dim}")

In [ ]:
mask_aca = metadata["superclass"] == "aca"
metadata = metadata[mask_aca]
embeddings = embeddings[mask_aca]

In [ ]:
y = metadata["class_name"].values
groups = metadata["patient_id"].values

In [ ]:
norms = np.linalg.norm(embeddings, axis=1)
print("Mean norm:", norms.mean())
print("Min norm:", norms.min(), "Max norm:", norms.max())

In [ ]:
cv = list(custom_balanced_group_kfold(
    embeddings,
    y,
    groups,
    n_splits=4,
))

In [ ]:
np.unique(y)

In [ ]:
train_idx, test_idx = cv[1]

In [ ]:
embeddings = normalize(embeddings, norm="l2")

In [ ]:
def remove_image_pcs_for_normalized(embeddings, image_ids, n_components=2):
    """
    Remove principal components that capture image-level variation,
    specially adapted for L2-normalized embeddings.
    
    Parameters:
    -----------
    embeddings : numpy array of shape (n_samples, n_features)
        L2-normalized embedding vectors
    image_ids : numpy array of shape (n_samples,)
        Image ID for each tile
    n_components : int
        Number of principal components to remove
    
    Returns:
    --------
    corrected_embeddings : numpy array of shape (n_samples, n_features)
        Embeddings with image-level PCs removed
    """
    unique_image_ids = np.unique(image_ids)
    
    # Compute image means
    image_means = np.zeros((len(unique_image_ids), embeddings.shape[1]))
    for i, img_id in enumerate(unique_image_ids):
        mask = image_ids == img_id
        # For L2-normalized vectors, take the mean and re-normalize
        mean_vector = embeddings[mask].mean(axis=0)
        image_means[i] = mean_vector / np.linalg.norm(mean_vector)
    
    # Compute PCA on image means to identify image-specific directions
    pca = PCA(n_components=n_components)
    pca.fit(image_means)
    image_pcs = pca.components_
    
    # For each embedding, remove projection onto image PCs
    corrected_embeddings = embeddings.copy()
    for i in range(len(embeddings)):
        embedding = embeddings[i]
        
        # Remove projections onto image PCs
        for pc in image_pcs:
            # Calculate projection
            proj = np.dot(embedding, pc) * pc
            # Subtract projection
            embedding = embedding - proj
            
        # Renormalize to unit length
        corrected_embeddings[i] = embedding / np.linalg.norm(embedding)
    
    return corrected_embeddings

# Apply to your normalized embeddings
image_ids = metadata["image_id"].values
embeddings = remove_image_pcs_for_normalized(embeddings, image_ids, n_components=8)

In [ ]:
def plot_elbow_method(embeddings, max_clusters=20):
    """Plot the elbow method to find optimal number of clusters."""
    inertia = []
    k_range = range(1, max_clusters + 1)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init="auto")
        kmeans.fit(embeddings)
        inertia.append(kmeans.inertia_)
    
    # Plot the elbow
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertia, 'o-', markersize=8)
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Inertia (Within-Cluster Sum of Squares)')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)
    
    # Calculate the angle/second derivative to find the elbow point
    angles = []
    for i in range(1, len(inertia)-1):
        x1, y1 = k_range[i-1], inertia[i-1]
        x2, y2 = k_range[i], inertia[i]
        x3, y3 = k_range[i+1], inertia[i+1]
        
        # Calculate the angle between the two line segments
        angle = np.abs(np.arctan2(y3-y2, x3-x2) - np.arctan2(y2-y1, x2-x1))
        angles.append((k_range[i], angle))
    
    # Find the point with maximum angle
    elbow_point = max(angles, key=lambda x: x[1])[0]
    plt.axvline(x=elbow_point, color='r', linestyle='--', label=f'Elbow point: k={elbow_point}')
    plt.legend()
    plt.show()
    
    return elbow_point

# Normalize embeddings before clustering
embeddings_norm = normalize(embeddings, norm="l2")
elbow_k = plot_elbow_method(embeddings_norm)

In [ ]:
# Fit k-means
n_clusters = 8  # you can tune this (e.g., 5-8 for LUAD patterns)
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init="auto")
cluster_labels = kmeans.fit_predict(embeddings)

# Store cluster assignments in metadata
metadata["cluster"] = cluster_labels

In [ ]:
metadata.head()

In [ ]:
# %%
# Get LUAD metadata for training and test
metadata_train = metadata.iloc[train_idx]
metadata_test = metadata.iloc[test_idx]


# Helper function to compute histograms per WSI
def compute_wsi_histograms(meta, n_clusters):
    wsi_histograms = {}
    wsi_labels = {}

    for image_id, group in meta.groupby("image_id"):
        cluster_counts = np.bincount(group["cluster"], minlength=n_clusters)
        histogram = cluster_counts / cluster_counts.sum()  # normalize
        wsi_histograms[image_id] = histogram
        wsi_labels[image_id] = group["class_name"].iloc[0]  # assuming consistent label

    return wsi_histograms, wsi_labels

# Compute histograms
train_histograms, train_labels = compute_wsi_histograms(metadata_train, n_clusters)
test_histograms, test_labels = compute_wsi_histograms(metadata_test, n_clusters)


In [ ]:
# %%
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Convert dicts to arrays
X_train = np.stack(list(train_histograms.values()))
y_train = np.array(list(train_labels.values()))

X_test = np.stack(list(test_histograms.values()))
y_test = np.array(list(test_labels.values()))

# Train logistic regression
clf = LogisticRegression(multi_class="multinomial", max_iter=1000)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)

print("Classification report:")
print(classification_report(y_test, y_pred))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
X_train.shape